In [1]:
import os 
from glob import glob
from concurrent.futures import ProcessPoolExecutor

def gdal_reproj(fi, fo, epsg_code=4326):
    if not os.path.isfile(fo):
        cmd = f"gdalwarp -q -t_srs EPSG:{epsg_code} {fi} {fo}"
        os.system(cmd)
    else:
        print("...")

In [2]:
ornl_tif_dir = "/media/ljp238/12TBWolf/ARCHIEVE/Brazil_ORNL_tif/meta_stateless/state"
ornl_dtm_dir = "/media/ljp238/12TBWolf/ARCHIEVE/Brazil_ORNL_tif/meta_stateless/reproj/dtm"
ornl_dsm_dir = "/media/ljp238/12TBWolf/ARCHIEVE/Brazil_ORNL_tif/meta_stateless/reproj/dsm"

dtm_files = glob(f"{ornl_tif_dir}/*DTM*.tif"); print(len(dtm_files))
dsm_files = glob(f"{ornl_tif_dir}/*DSM*.tif"); print(len(dsm_files))


idtm_files = glob(f"{ornl_dtm_dir}/*.tif"); print(len(idtm_files))
idsm_files = glob(f"{ornl_dsm_dir}/*.tif"); print(len(idsm_files))

3111
3110
3111
0


ORNL reproj

In [3]:

os.makedirs(ornl_dtm_dir, exist_ok=True)
os.makedirs(ornl_dsm_dir, exist_ok=True)
epsg_code=4326
files = dsm_files
outdir = ornl_dsm_dir

In [4]:
if __name__ == "__main__":
    with ProcessPoolExecutor(50) as pex:
        for fi in files:
            fo = os.path.join(outdir, os.path.basename(fi))
            pex.submit(gdal_reproj,fi,fo,epsg_code)


In [5]:
dtm_files = glob(f"{ornl_tif_dir}/*DTM*.tif"); print(len(dtm_files))
dsm_files = glob(f"{ornl_tif_dir}/*DSM*.tif"); print(len(dsm_files))

idtm_files = glob(f"{ornl_dtm_dir}/*.tif"); print(len(idtm_files))
idsm_files = glob(f"{ornl_dsm_dir}/*.tif"); print(len(idsm_files))

3111
3110
3111
3110


In [ ]:
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.features import shapes
from shapely.geometry import shape
from concurrent.futures import ProcessPoolExecutor

def extract_valid_shapes(tif_file):
    """Extracts valid (non-nodata) regions as polygons from a TIFF file."""
    with rasterio.open(tif_file) as src:
        band = src.read(1)
        mask = band != src.nodata  # Mask valid data
        
        # Extract polygons
        return [shape(geom) for geom, _ in shapes(band, mask=mask, transform=src.transform)]

def process_tifs_parallel(tif_files):
    """Processes TIFFs in parallel and extracts valid polygons."""
    polygons = []
    with ProcessPoolExecutor() as executor:
        results = executor.map(extract_valid_shapes, tif_files)
        for result in results:
            polygons.extend(result)  # Flatten list
    return polygons

def save_to_gpkg(polygons, gpkg_output, crs):
    """Saves polygons to a GeoPackage."""
    gdf = gpd.GeoDataFrame(geometry=polygons, crs=crs)
    gdf.to_file(gpkg_output, driver="GPKG")
    print(f'saving...\n{gpkg_output}')

def tileindex(tif_files, gpkg_output):
    """Main function to extract valid regions from TIFFs and save to GeoPackage."""
    if not tif_files:
        raise ValueError("No TIFF files provided.")

    with rasterio.open(tif_files[0]) as src:
        crs = src.crs  # Use CRS from the first file

    polygons = process_tifs_parallel(tif_files)
    save_to_gpkg(polygons, gpkg_output, crs)




/home/ljp238/miniconda3/envs/samlidar/lib/python3.9/site-packages/pyproj/__init__.py:95: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [11]:
def gdaltindex(fo, dir):
    cmd = f"gdaltindex -f GPKG {fo} {dir}/*.tif"
    os.system(cmd)

In [7]:
ornl_rerpj_dir = "/media/ljp238/12TBWolf/ARCHIEVE/Brazil_ORNL_tif/meta_stateless/reproj"
dtm_gpkg = os.path.join(ornl_rerpj_dir, 'dtm.gpkg')
dsm_gkpg = os.path.join(ornl_rerpj_dir, 'dsm.gpkg')


In [13]:
gdaltindex(dsm_gkpg, ornl_dsm_dir)

In [14]:
gdaltindex(dtm_gpkg, ornl_dtm_dir)

In [15]:
import geopandas as gpd 

In [18]:
#g = gpd.read_file(dsm_gkpg)

In [ ]:
# create mosaic by closer things

'/media/ljp238/12TBWolf/ARCHIEVE/Brazil_ORNL_tif/meta_stateless/reproj/dsm'

In [ ]:



# Example usage
tif_files =  idtm_files# ["file1.tif", "file2.tif", "file3.tif"]
gpkg_output = dtm_gpkg
#tileindex(tif_files, gpkg_output)

In [10]:
tif_files =  idsm_files# ["file1.tif", "file2.tif", "file3.tif"]
gpkg_output = dsm_gkpg
#tileindex(tif_files, gpkg_output)

In [19]:
import rasterio
from shapely.geometry import box

# Function to get the bounding box of a TIFF file
def get_bounding_box(tif_file):
    with rasterio.open(tif_file) as src:
        bounds = src.bounds  # Returns (left, bottom, right, top)
        return box(bounds.left, bounds.bottom, bounds.right, bounds.top)

In [20]:


bounding_boxes = [get_bounding_box(tif) for tif in tif_files]

In [38]:
from sklearn.cluster import DBSCAN
import numpy as np

# Convert bounding boxes to centroids (geographic center points)
centroids = [(bbox.centroid.x, bbox.centroid.y) for bbox in bounding_boxes]

# Apply DBSCAN clustering
eps = 1.2#0.9 # Maximum distance between points to be considered in the same cluster (in degrees)
min_samples = 2#2  # Minimum number of points to form a cluster
db = DBSCAN(eps=eps, min_samples=min_samples).fit(centroids)

# Get cluster labels
labels = db.labels_
print(np.unique(labels))
print(print(len(np.unique(labels))))

# Group files by cluster
clusters = {}
for i, label in enumerate(labels):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(tif_files[i])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
17
None


In [27]:
np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22])

In [46]:
import rasterio
from shapely.geometry import box
from sklearn.cluster import DBSCAN
import subprocess

# Step 1: Function to get the bounding box of a TIFF file
def get_bounding_box(tif_file):
    with rasterio.open(tif_file) as src:
        bounds = src.bounds  # Returns (left, bottom, right, top)
        return box(bounds.left, bounds.bottom, bounds.right, bounds.top)

# Step 2: Function to set nodata value for a TIFF file
def set_nodata_value(input_file, nodata_value=-9999):
    with rasterio.open(input_file, 'r+') as src:
        src.nodata = nodata_value

# Step 3: Function to merge TIFF files using GDAL with nodata set to -9999
def merge_tiffs(input_files, output_file, nodata_value=-9999):
    command = [
        "gdal_merge.py",
        "-o", output_file,
        "-a_nodata", str(nodata_value),  # Set nodata value
        "-of", "GTiff"                  # Output format (GeoTIFF)
    ] + input_files
    subprocess.run(command)

# Step 4: Main Workflow
def group_and_merge_tifs(outdir, tif_files, eps=0.1, min_samples=2, nodata_value=-9999):
    # Step 4.1: Extract bounding boxes and centroids
    bounding_boxes = [get_bounding_box(tif) for tif in tif_files]
    centroids = [(bbox.centroid.x, bbox.centroid.y) for bbox in bounding_boxes]

    # Step 4.2: Apply DBSCAN clustering
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(centroids)
    labels = db.labels_
    print(np.unique(labels))
    print('Ulabels',print(len(np.unique(labels))))

    # Step 4.3: Group files by cluster
    clusters = {}
    for i, label in enumerate(labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(tif_files[i])

    # Step 4.4: Standardize nodata values for all input files
    for tif_file in tif_files:
        set_nodata_value(tif_file, nodata_value=nodata_value)

    # Step 4.5: Merge files in each cluster
    for cluster_id, files in clusters.items():
        if cluster_id != -1:  # Ignore noise points (-1 label)
            output_file = f"{outdir}/merged_cluster_{cluster_id}.tif"
            merge_tiffs(files, output_file, nodata_value=nodata_value)
            print(f"Merged {len(files)} files into {output_file}")

# # Example usage
# if __name__ == "__main__":
#     # List of your TIFF files


#     # Run the workflow
#     group_and_merge_tifs(
#         tif_files=tif_files,
#         eps=0.1,               # Maximum distance between points to be considered in the same cluster (in degrees)
#         min_samples=2,         # Minimum number of files required to form a cluster
#         nodata_value=-9999     # Nodata value for merged files
#     )

In [47]:
ornl_dsm_cluster_dir = "/media/ljp238/12TBWolf/ARCHIEVE/Brazil_ORNL_tif/meta_stateless/reproj/dsm_cluster"
os.makedirs(ornl_dsm_cluster_dir, exist_ok=True)

group_and_merge_tifs(outdir=ornl_dsm_cluster_dir, tif_files=idsm_files, eps=1.2, min_samples=2, nodata_value=-9999)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
17
Ulabels None
0...10...20...30...40...50...60...70...80...90...100 - done.
Merged 891 files into /media/ljp238/12TBWolf/ARCHIEVE/Brazil_ORNL_tif/meta_stateless/reproj/dsm_cluster/merged_cluster_0.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Merged 531 files into /media/ljp238/12TBWolf/ARCHIEVE/Brazil_ORNL_tif/meta_stateless/reproj/dsm_cluster/merged_cluster_1.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Merged 133 files into /media/ljp238/12TBWolf/ARCHIEVE/Brazil_ORNL_tif/meta_stateless/reproj/dsm_cluster/merged_cluster_2.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Merged 45 files into /media/ljp238/12TBWolf/ARCHIEVE/Brazil_ORNL_tif/meta_stateless/reproj/dsm_cluster/merged_cluster_3.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Merged 29 files into /media/ljp238/12TBWolf/ARCHIEVE/Brazil_ORNL_tif/meta_stateless/reproj/dsm_cluster/merged_cluster_4.tif
0..

EBA repro